In [10]:
# Path setup to resolve package imports (dynamic project root)
import sys, os

# Derive project root from current notebook directory: <project>/notebook
notebook_dir = os.getcwd()
project_root = os.path.dirname(notebook_dir)

if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [ ]:
from src.agents.tools.db_tools import db_tool_manager

tools = db_tool_manager.get_tools()
print(f"✅ {len(tools)} tools ready")
print("Tool names:", [t.name for t in tools])


✅ 4 tools ready
Tool names: ['list_all_tables', 'get_table_schema', 'preview_sql', 'execute_sql']


In [ ]:
# Cell 1: COMPLETE STREAMING WITH FINAL RESULT
from langchain.agents import create_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain_openai import ChatOpenAI
from src.config.prompt import system_prompt
from langchain_core.messages import HumanMessage

checkpointer = MemorySaver()
gptLLM = ChatOpenAI(model_name="gpt-4", temperature=0)

agent = create_agent(
    model=gptLLM,
    tools=tools,
    system_prompt=system_prompt,
    checkpointer=checkpointer
)

def stream_complete(question):
    """Stream TOOLS + REASONING + FINAL RESULT"""
    config = {"configurable": {"thread_id": "complete"}}
    steps = []
    
    print("🤖 COMPLETE STREAMING")
    print("━" * 60)
    
    full_content = ""
    
    for chunk in agent.stream(
        {"messages": [HumanMessage(content=question)]},
        config,
        stream_mode="values"
    ):
        if "messages" in chunk and chunk["messages"]:
            msg = chunk["messages"][-1]
            
            # TOOL CALLS (Step 1)
            if hasattr(msg, 'tool_calls') and msg.tool_calls:
                tool_call = msg.tool_calls[0]
                print(f"🟡 TOOL: {tool_call['name']}({tool_call['args']})")
                steps.append(tool_call['name'])
            
            # TOOL RESULTS (Step 2)  
            elif msg.type == "tool":
                print(f"🟢 RESULT: {msg.content[:100]}...")
            
            # FINAL REASONING + ANSWER (Step 3)
            elif msg.content:
                full_content += msg.content
                print(msg.content, end="", flush=True)
    
    print(f"\n\n🎯 **FINAL RESULT**")
    print(f"📊 Steps: {len(steps)} | Content: {len(full_content)} chars")
    print("-" * 60)
    print(full_content.strip())

# TEST IT
stream_complete("Employees in Sales department HumanResources")

🤖 COMPLETE STREAMING
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Employees in Sales department HumanResources🟡 TOOL: find_relevant_tables({'question': 'Employees in Sales department HumanResources'})
🟢 RESULT: Error: find_relevant_tables is not a valid tool, try one of [list_all_tables, get_table_schema, prev...
🟡 TOOL: list_all_tables({})
🟢 RESULT: dbo.AWBuildVersion, dbo.DatabaseLog, dbo.ErrorLog, Sales.CountryRegionCurrency, Sales.CreditCard, Sa...
🟡 TOOL: get_table_schema({'table_names': 'HumanResources.Employee, HumanResources.Department, HumanResources.EmployeeDepartmentHistory'})
🟢 RESULT: 
CREATE TABLE [HumanResources].[Department] (
	[DepartmentID] SMALLINT NOT NULL IDENTITY(1,1), 
	[Mo...
🟡 TOOL: preview_sql({'sql': "SELECT TOP 5 E.BusinessEntityID, E.JobTitle FROM HumanResources.Employee E JOIN HumanResources.EmployeeDepartmentHistory EDH ON E.BusinessEntityID = EDH.BusinessEntityID WHERE EDH.DepartmentID = (SELECT DepartmentID FROM HumanResources.Department